In [2]:
from importlib.metadata import version

print("matplotlib version:", version("matplotlib"))
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

matplotlib version: 3.7.5
torch version: 2.4.1
tiktoken version: 0.7.0


In [7]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "dropout_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [ ]:
import torch
import torch.nn as nn

class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
    
    def forward(self, x):
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()

    def forward(self, x):
        return x


class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["dropout_rate"])

        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=True
        )
    
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits    

In [8]:
### test using an example
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [ ]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print(logits.shape)
print(logits)  # batch size = 2, window size = 4, vocab size = 50257

torch.Size([2, 4, 50257])
tensor([[[-1.4263,  0.1865, -1.0108,  ..., -1.8458, -0.1784, -0.3773],
         [-0.1230,  0.3998, -0.0121,  ..., -0.4483,  1.5452,  1.2868],
         [ 0.6295,  1.9302, -0.3573,  ...,  1.6177,  0.2114,  0.6670],
         [ 0.8993,  1.6951, -0.8083,  ...,  1.0100, -0.7267, -0.7965]],

        [[-1.4560,  0.5308, -0.7231,  ..., -1.3773, -0.2287, -0.4222],
         [-0.6794,  1.0382,  0.0130,  ..., -0.2712,  0.7241,  1.5719],
         [ 0.6346,  1.0774, -0.1682,  ..., -0.5877, -0.0291,  0.4476],
         [ 0.0516, -0.8008,  0.6667,  ...,  2.0342,  0.0381, -0.0332]]],
       grad_fn=<ViewBackward0>)


### Normalization

In [10]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5 # avoid var = 0
        self.scale = nn.Parameter(torch.ones(emb_dim))  # trainable, linear
        self.shift = nn.Parameter(torch.zeros(emb_dim)) # trainable, linear

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x-mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

### GELU